In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import itertools
from collections import OrderedDict
from functools import partial

from sklearn.datasets import load_svmlight_file, make_circles


from links import LinksClassifier
from logit import LogisticRegressionPairwise, LogisticRegression


from sklearn.model_selection import ParameterGrid, StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook as tqdm

from new_experiment_runner.cacher import CSVCacher

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:
%autoreload

In [60]:
datafiles_toy = [
    r'data/diabetes_scale.libsvm',
    r'data/breast-cancer_scale.libsvm',
]

In [79]:
datasets = OrderedDict([(os.path.split(f)[-1].replace('.libsvm', ''),
                             load_svmlight_file(f))
                            for f in datafiles_toy])

In [80]:
datasets['circles'] = make_circles(n_samples=400, noise=0.1, factor=0.51)
datasets['moons'] = make_circles(n_samples=400, noise=0.1)

In [5]:
results = pd.read_csv('data/results_sens.csv')

In [6]:
results.shape, results.columns

((900, 5),
 Index([u'k', u'method', u'dataset', u'loss', u'train_score'], dtype='object'))

In [10]:
results.head()

,k,method,dataset,loss,train_score
0,0.0,zeros,diabetes_scale,56.357508,0.695652
1,1.0,zeros,diabetes_scale,56.357508,0.695652
2,2.0,zeros,diabetes_scale,56.357508,0.695652
3,3.0,zeros,diabetes_scale,56.357508,0.695652
4,4.0,zeros,diabetes_scale,56.357508,0.695652


In [20]:
df = pd.groupby(results, by=['dataset', 'method']).agg({'loss': [np.std]}).unstack(level=-1)

In [23]:
df.T


dataset                            circles  diabetes_scale         moons
         method                                                         
loss std normal               4.548139e-08    6.684074e-09  7.496055e-09
         normal_multivariate  1.652018e-08    7.265744e-09  5.570900e-09
         normal_univariate    3.075394e-08    7.646498e-09  2.445021e-09
         random_labels        9.307598e-09    2.440747e-09  6.107176e-09
         random_links_diff    2.116533e-08    3.410780e-09  7.626612e-09
         zeros                0.000000e+00    5.206897e-10  0.000000e+00

In [24]:
with open('data\start.tex', 'w') as f: 
     df.T.to_latex(f)